In [42]:
import pandas as pd 
import os
import glob
import difflib

In [43]:
n=5

mdf=pd.read_excel(f'/workspaces/vikrant_dubai/RAW/SHJ/SHJ {n}.xlsx')
excel_file = pd.ExcelFile(f'/workspaces/vikrant_dubai/RAW/SHJ/SHJ {n}.xlsx')

df = pd.DataFrame(columns=["Station Name", "Latitude", "Longitude", "Region", "Area Classification"])
dfs = []
for i in range(len(excel_file.sheet_names)):
    df = pd.read_excel(f"/workspaces/vikrant_dubai/RAW/SHJ/SHJ {n}.xlsx", sheet_name = excel_file.sheet_names[i])
    df=df.drop([0,1,2,3,4,5])
    df=df.iloc[:,:13]


    mapped_data = {
        "Station Name": df.iloc[0, 2],  
        "Latitude": df.iloc[1, 4],  
        "Longitude": df.iloc[1, 7],  
        "Region": df.iloc[0, 6],  
        "Area Classification": df.iloc[0, 11], 
        # "Emission Source": df.iloc[0, 8],
        "Station Operation Year": df.iloc[1, 11] 
    }

    temp_df = pd.DataFrame([mapped_data])
    dfs.append(temp_df)

final_df = pd.concat(dfs, ignore_index=True)

# # final_df.to_csv(f'/workspaces/vikrant_dubai/final/details{k}.csv', index=False)

In [44]:
for mon in range(1,13):
   
    n=mon

    mdf=pd.read_excel(f'/workspaces/vikrant_dubai/RAW/SHJ/SHJ {n}.xlsx')
    excel_file = pd.ExcelFile(f'/workspaces/vikrant_dubai/RAW/SHJ/SHJ {n}.xlsx')

    columns_of_main_df = ['Station Name','Date', 'Time', 'SO2', 'Data Cap. (SO2)', 'SO2 Avg', 'NO2', 'Data Cap. (NO2)', 'NO2 Avg',      'O3', 'Data Cap. (O3)', 'O3 8H Avg.', 'Daily Capture O3', 'O3 Daily Average', 'O3 Daily Average (NS)', 'CO',  'Data Cap. (CO)', 'CO 8H Avg.', 'Daily Capture CO', 'CO Daily Average', 'PM10',    'Data Cap. (PM10)', 'PM10 Avg', 'PM2.5', 'Data Cap. (PM2.5)', 'PM2.5 Avg', 'Lower Ambient Temperature', 'Upper Ambient Temperature', 'Barometric Pressure', 'Relative Humidity', 'Wind Direction', 'Wind Speed', 'H2S', 'Toluene', 'O-Xylene', 'Ethylbenzene', 'MP-Xylene', 'Benzene', 'CH4', 'NMHC', 'THC', 'Noise', 'Solar Radiation','Atm P.']
    for s in range(len(excel_file.sheet_names)):
        main_df = pd.DataFrame(columns=columns_of_main_df)

        # Read the Excel file and preprocess the data
        mdf = pd.read_excel(f'/workspaces/vikrant_dubai/RAW/SHJ/SHJ {n}.xlsx', sheet_name=excel_file.sheet_names[s])
        mdf = mdf.drop([0, 1, 2, 3,4,5,6,7,8,9,10]).reset_index(drop=True)
        mdf.columns = mdf.iloc[0]
        mdf = mdf[1:].reset_index(drop=True).drop([0, 1]).reset_index(drop=True)


        # Convert 'Date/Time' column to separate 'Date' and 'Time' columns
        mdf['Date/Time'] = pd.to_datetime(mdf['Date/Time'], format='%d/%m/%Y %H:%M', dayfirst=True) 
        mdf['Date'] = mdf['Date/Time'].dt.date
        mdf['Time'] = mdf['Date/Time'].dt.time
        new_df_columns = mdf.columns.tolist()

        mdf.columns = new_df_columns

        # Reorder columns and drop the original 'Date/Time' column
        mdf = mdf[['Date', 'Time'] + [col for col in mdf.columns if col not in ['Date', 'Time']]]
        mdf = mdf.drop(columns=['Date/Time'])

        # add column station name
        mdf['Station Name'] = excel_file.sheet_names[s]

        # Format the 'Date' column
        mdf['Date'] = pd.to_datetime(mdf['Date']).dt.strftime('%d/%m/%Y')


        # List of values to find matches for
        values = [str(col).upper() for col in mdf.columns]

        # List of column names
        l1 = [str(col).upper() for col in main_df.columns]

        # Loop through values and find the closest match
        matches = {}
        for value in values:
            closest_match = difflib.get_close_matches(value, l1, n=1)
            if closest_match:
                main_df[main_df.columns[main_df.columns.str.upper() == closest_match[0]][0]] = mdf[mdf.columns[mdf.columns.str.upper() == value][0]]
            else:
                main_df[value] = ''


        main_df.to_csv(f'/workspaces/vikrant_dubai/Transformed/{excel_file.sheet_names[s]}.csv', index=False)
            
    # Get a list of all CSV files in the transformed folder
    csv_files = [f for f in os.listdir('/workspaces/vikrant_dubai/Transformed') if f.endswith('.csv')]

    # Read and concatenate all CSV files
    combined_df = pd.concat([pd.read_csv(os.path.join('/workspaces/vikrant_dubai/Transformed', f)) for f in csv_files], ignore_index=True)

    # Save the combined DataFrame to a new CSV file
    combined_df.to_csv(f'/workspaces/vikrant_dubai/final/SHJ/SHJ {n}.csv', index=False)


    # Get a list of all files in the Transformed folder
    files = glob.glob('/workspaces/vikrant_dubai/Transformed/*')

    # Iterate over the list of files and remove each one
    for f in files:
        os.remove(f)

In [45]:
mdf=pd.read_excel(f'/workspaces/vikrant_dubai/RAW/SHJ/SHJ {n}.xlsx')
mdf = mdf.drop([0, 1, 2, 3,4,5,6,7,8,9,10]).reset_index(drop=True)
mdf.columns = mdf.iloc[0]
mdf = mdf[1:].reset_index(drop=True).drop([0, 1]).reset_index(drop=True)

In [46]:
mdf

,NaN,Date/Time,SO2,NO2,O3,NaN,CO,NaN,PM10,PM2.5,...,H2S,Toluene,O_Xylene,EthylBenzene,mp_xylene_,Benzene,CH2,NMHC,THC,Noise
0,1,2023-12-01 00:00:00,13.16,44.51,8.85,32.8275,1.541,0.598,55,NaN,...,5.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2023-12-01 01:00:00,8.88,42.73,4.96,24.9175,0.246,0.601625,55.8,NaN,...,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2023-12-01 01:59:59.990000,9.49,40.14,2.69,17.41125,0.206,0.55625,58.1,NaN,...,1.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2023-12-01 02:59:59.985000,10.94,38.32,3.76,11.81,0.249,0.502875,57.5,NaN,...,2.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2023-12-01 03:59:59.980000,11.76,38.45,7.78,7.65125,0.629,0.53,58.5,NaN,...,4.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,764,2023-12-31 18:59:56.305000,27.27,10.23,75.9,89.7575,0.022,0.509,59.4,NaN,...,3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
740,765,2023-12-31 19:59:56.300000,25.08,20.98,48.09,83.52875,0.049,0.4735,64.6,NaN,...,3.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,766,2023-12-31 20:59:56.295000,18.47,25.76,31.22,76.7125,0.029,0.40975,73.8,NaN,...,2.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,767,2023-12-31 21:59:56.290000,16.07,36.22,8.92,67.43,0.102,0.25025,78.3,NaN,...,1.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
